In [1]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1015. Fire Sufficient Similarity/P1015.3. Analyses/P1015.3.2. Data Processing/Output'
cur_date = '031825'

library(missForest)
library(readxl)
library(openxlsx)
library(tidyverse)
library(imputeLCMD)
library(factoextra)

# reading in files
ws_df = data.frame(Data = 'WS', read_excel("Input/Woodsmoke_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)
wf_df = data.frame(Data = 'WF', read_excel("Input/Wildfire_Data_012825.xlsx", sheet = 2)) %>%
    select(-Sample_Number)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
an

NOTES TO SELF:
- change 'replicate' to 'instance' and 'chemical class' to 'class'

In [2]:
head(ws_df)
head(wf_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,1263480,Burnet et al. 1990,NA,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WF,1289821,Liang et al. 2021,1,PAH,Volume,NA,1-(10-methylanthracen-9-yl)ethanone,5
2,WF,1289821,Liang et al. 2021,1,PAH,Volume,DTXSID50176885,1-Acenaphthenone,1
3,WF,1289737,Campbell et al. 2024,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
4,WF,1289739,Campos et al. 2019,1,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
5,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA
6,WF,1289739,Campos et al. 2019,NA,PAH,Weight,DTXSID1074759,1-Methylchrysene,NA


In [3]:
dim(ws_df)
dim(wf_df)

[1] 6607    9

[1] 4340    9

# Filter #1

Removing duplicate records for instances that weren't measured at all (MAR). 

In [4]:
`%notin%` <- Negate(`%in%`)

ws_df = ws_df %>%
    filter(Replicate %notin% NA)

wf_df = wf_df %>%
    filter(Replicate %notin% NA)

In [5]:
dim(ws_df)
dim(wf_df)

[1] 4203    9

[1] 2378    9

In [6]:
# combining dfs
combined_df = rbind(ws_df, wf_df)
head(combined_df)

,Data,HAWC_ID,Study,Replicate,Chemical_Class,Metric,DTXSID,Name,Value
,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,11.3
2,WS,821855,Erlandsson et al. 2020,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,2.2999999999999998
3,WS,1257056,McCarrick et al. 2024,1,PAH,Weight,DTXSID3074787,1-Methylanthracene,1.66
4,WS,267140,Alfheim and Ramdahl 1984,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
5,WS,1263480,Burnet et al. 1990,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA
6,WS,267091,Forchhammer et al. 2012,1,PAH,Volume,DTXSID3074787,1-Methylanthracene,NA


In [7]:
length(unique(combined_df$Name))

[1] 192

192 unique metals/chemicals.

# Filter #2: Chemical/Metal Background Filters

Determining what variables have missing data for all variables delineated based on the dfs the data will be split into and analyzed. 

A variable (ie. `Name` which represents a chemical or metal) will be retained if >= 50% of its experimental data points were measurable values. In other words, chemicals were excluded if they were NAs (not evaluated) or had non-detect values across > 50% of the samples.

In [8]:
sample_type_presence_df = combined_df %>%
    # # basing counts off of the mean values
    # mutate(Avg_Value = mean(as.numiValue)) %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 50% experimental values for each chemical/metal
    group_by(Metric, Chemical_Class, Data, Name) %>%
    # calculating the percentage of experimental data points
    reframe(Variable_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Variable_Presence_Percentage >= 50) 

head(sample_type_presence_df)

Metric,Chemical_Class,Data,Name,Variable_Presence_Percentage
<chr>,<chr>,<chr>,<chr>,<dbl>
Volume,Metal,WF,As,100
Volume,Metal,WF,Ca,100
Volume,Metal,WF,Cd,100
Volume,Metal,WF,Co,100
Volume,Metal,WF,Fe,100
Volume,Metal,WF,Ga,100


In [9]:
# filtering the original df
filter1_df = inner_join(sample_type_presence_df[,1:4], combined_df) %>%
    # metals don't have a DTXSID, so making that col their name
    mutate(DTXSID = ifelse(DTXSID != 'NA', DTXSID, Name)) 

head(filter1_df)

Joining with `by = join_by(Metric, Chemical_Class, Data, Name)`


Metric,Chemical_Class,Data,Name,HAWC_ID,Study,Replicate,DTXSID,Value
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,WF,As,1070627,Siregar et al. 2022,1,As,2.2999999999999998
Volume,Metal,WF,As,1070627,Siregar et al. 2022,2,As,0.8
Volume,Metal,WF,As,1289884,Samsonova et al. 2012,1,As,67
Volume,Metal,WF,As,1289884,Samsonova et al. 2012,2,As,117
Volume,Metal,WF,As,1289884,Samsonova et al. 2012,3,As,183
Volume,Metal,WF,As,1289884,Samsonova et al. 2012,4,As,50


In [10]:
dim(combined_df)
dim(filter1_df)

[1] 6581    9

[1] 4225    9

Started with 6581 records, 2392 were removed, leaving 4189. 

# Filter 3: Study Background Filter
WORDS

In [11]:
study_type_presence_df = filter1_df %>%
    # # basing counts off of the mean values
    # mutate(Avg_Value = mean(Value)) %>%
    # if an experimental value is there count it as being "present"
    mutate(count = ifelse(Value %notin% c('NA', 'ND'), 1, 0)) %>%
    # determining which have at least 50% experimental values for each chemical/metal
    group_by(Metric, Chemical_Class,  HAWC_ID) %>%
    # calculating the percentage of experimental data points
    reframe(Study_Presence_Percentage = (sum(count)/n()) * 100) %>%
    filter(Study_Presence_Percentage >= 50) 

head(study_type_presence_df)

Metric,Chemical_Class,HAWC_ID,Study_Presence_Percentage
<chr>,<chr>,<dbl>,<dbl>
Volume,Metal,266980,100
Volume,Metal,266987,100
Volume,Metal,452289,100
Volume,Metal,885041,100
Volume,Metal,993956,100
Volume,Metal,1070627,100


In [12]:
filter2_df = inner_join(study_type_presence_df[,1:3], filter1_df)

head(filter2_df)

Joining with `by = join_by(Metric, Chemical_Class, HAWC_ID)`


Metric,Chemical_Class,HAWC_ID,Data,Name,Study,Replicate,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,1,K,9700
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,2,K,8800
Volume,Metal,266980,WS,Pb,Stockfelt et al. 2012,1,Pb,170
Volume,Metal,266980,WS,Pb,Stockfelt et al. 2012,2,Pb,400
Volume,Metal,266980,WS,Rb,Stockfelt et al. 2012,1,Rb,73
Volume,Metal,266980,WS,Rb,Stockfelt et al. 2012,2,Rb,105


In [13]:
dim(filter1_df)
dim(filter2_df)

[1] 4225    9

[1] 4145    9

MORE WORDS!!

# Filter #3: Retaining Metals/or Chemicals that are Present in Both WS/WF Samples

In the intial filter, WS and WF samples were combined. However, this time variables (ie. metal or chemical) will be split based on their `Metric` (ie. volume or weight) and then retained if that variable is found in both woodsmoke and wildfire samples within volume or weight samples.

In [14]:
split_filtered_df = filter2_df %>%
    group_by(Data, Metric) %>%
    group_split

split_ws_vol_df = split_filtered_df[[3]]
split_ws_weight_df = split_filtered_df[[4]]
split_wf_vol_df = split_filtered_df[[1]]
split_wf_weight_df = split_filtered_df[[2]]

In [15]:
# seeing how many unique variables are in each df and if they're consistent in each file
length(unique(split_ws_vol_df$Name))
length(unique(split_wf_vol_df$Name))
length(unique(split_ws_weight_df$Name))
length(unique(split_wf_weight_df$Name))

[1] 95

[1] 68

[1] 66

[1] 70

In [16]:
# they're not so first getting variables that are in weight or volume samples
consistent_wf_vol_df = split_wf_vol_df %>%
    filter(Value != 'NA', Name %in% unique(split_ws_vol_df$Name))
consistent_ws_vol_df = split_ws_vol_df %>%
    filter(Value != 'NA', Name %in% consistent_wf_vol_df$Name)
consistent_wf_weight_df = split_wf_weight_df %>%
    filter(Value != 'NA', Name %in% unique(split_ws_weight_df$Name))
consistent_ws_weight_df = split_ws_weight_df %>%
    filter(Value != 'NA', Name %in% consistent_wf_weight_df$Name)

length(unique(consistent_wf_vol_df$Name))
length(unique(consistent_ws_vol_df$Name))
length(unique(consistent_wf_weight_df$Name))
length(unique(consistent_ws_weight_df$Name))

[1] 32

[1] 32

[1] 23

[1] 23

There were 98, 68, 94 and 70 woodsmoke weight, woodsmoke volume, wildfire weight, and wildfire volume samples, respectively. 33 variables were common between volume samples and 34 were common between weight records and will be retained.

In [17]:
# recombining data
vol_df = rbind(consistent_ws_vol_df, consistent_wf_vol_df)
weight_df = rbind(consistent_ws_weight_df, consistent_wf_weight_df)

head(vol_df)

Metric,Chemical_Class,HAWC_ID,Data,Name,Study,Replicate,DTXSID,Value
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,1,K,9700
Volume,Metal,266980,WS,K,Stockfelt et al. 2012,2,K,8800
Volume,Metal,266980,WS,Rb,Stockfelt et al. 2012,1,Rb,73
Volume,Metal,266980,WS,Rb,Stockfelt et al. 2012,2,Rb,105
Volume,Metal,266987,WS,Ca,Seagrave 2005,1,Ca,49200
Volume,Metal,266987,WS,Cd,Seagrave 2005,1,Cd,3500


# QRILC Imputation

Imputing non-detect data using QRILC didn't work, so I'm just replacing the non-detect data with zeros.????

In [18]:
# determining number of ND and NA values in each df
vol_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

weight_df %>%
    filter(Value %in% c('NA', 'ND')) %>%
    group_by(Value) %>%
    summarize(n())

Value,n()
<chr>,<int>


Value,n()
<chr>,<int>
ND,18


In [19]:
dim(vol_df)
dim(weight_df)

[1] 741   9

[1] 1053    9

In [20]:
# reformatting the volume df for export???
vol_df$Value = as.numeric(vol_df$Value)
# reordering cols????
vol_df = vol_df[,c(3,1,5:8,4,9)]

head(vol_df)

HAWC_ID,Metric,Name,Study,Replicate,DTXSID,Data,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>
266980,Volume,K,Stockfelt et al. 2012,1,K,WS,9700
266980,Volume,K,Stockfelt et al. 2012,2,K,WS,8800
266980,Volume,Rb,Stockfelt et al. 2012,1,Rb,WS,73
266980,Volume,Rb,Stockfelt et al. 2012,2,Rb,WS,105
266987,Volume,Ca,Seagrave 2005,1,Ca,WS,49200
266987,Volume,Cd,Seagrave 2005,1,Cd,WS,3500


Only the weight dataframe had non-detect values representing MNAR data. The remaining MAR data from the weight df will be imputed using random forest (RF).

In [21]:
mar_weight_df = weight_df %>%
    filter(Value == 'NA')

preimputed_df = anti_join(weight_df, mar_weight_df) 
preimputed_df$Value = as.numeric(preimputed_df$Value)
mar_weight_df$Value = as.numeric(mar_weight_df$Value)

# reordering cols
preimputed_df = preimputed_df[,c(3,1,4,6,7,2,8,5,9)]

head(preimputed_df)

Joining with `by = join_by(Metric, Chemical_Class, HAWC_ID, Data, Name, Study,
Replicate, DTXSID, Value)`
Warning message:
“NAs introduced by coercion”


HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cr,Cr,21.4
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cr,Cr,21.3
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Cu,Cu,65.6
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Cu,Cu,34.4
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Fe,Fe,1323.0
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Fe,Fe,1773.0


Gave up on QRILC imputation for all MNAR data, since I didn't have enough instances to impute within each study. I also couldn't impute across the entire dataset given that the number of data instances for each study wasn't the same. Therefore, sample groups were assigned a value of 0 if they had two or fewer experimental instances.

In [22]:
# creating a df for mnar data
mnar_df = preimputed_df %>%
    filter(is.na(Value)) 

mnar_df[c(1:13,18),9] = 0
head(mnar_df)

HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
267105,Weight,WS,Danielsen et al. 2011,1,Metal,Mn,Mn,0
267105,Weight,WS,Danielsen et al. 2011,2,Metal,Mn,Mn,0
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ca,Ca,0
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Fe,Fe,0
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Mn,Mn,0
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Mn,Mn,0
1040882,Weight,WS,Nordin et al. 2015,1,Metal,Ni,Ni,0
1040882,Weight,WS,Nordin et al. 2015,2,Metal,Ni,Ni,0
1063054,Weight,WS,Corsini et al. 2013,2,Metal,Cr,Cr,0


In [23]:
# adding these changed values into the preimputed df
preimputed_df = rbind(anti_join(preimputed_df, mnar_df[,1:8]),
      mnar_df)

Joining with `by = join_by(HAWC_ID, Metric, Data, Study, Replicate,
Chemical_Class, DTXSID, Name)`


In [24]:
QRILC_imputation = function(dataset){
    # """
    # Creating a quantile normalization function to normalize each sample.
    # :param (input): exposed and unexposed (vehicle) dfs
    # :output: 1 quantile normalized df
    # """
    #ADD WORDS
    set.seed(12)
    
    wider_dataset = dataset %>% 
        # removing these cols temporarily 
        select(-c("Chemical_Class", "Name")) %>%
        pivot_wider(names_from = DTXSID, values_from = Value)
   
    # normalizing data since that what the QRILC function wants
    # had to pseudo log transform to prevent Inf values
    QRILC_prep = wider_dataset[,6:ncol(wider_dataset)] %>%
         mutate_all(., function(x) log10(x + 1)) %>%
         as.matrix()
                        
    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log10_df = data.frame(imputed_QRILC_object[1]) 
    
    # converting back the original scale
    QRILC_df = QRILC_log10_df %>%
        mutate_all(., function(x) 10^x - 1)
    
    imputed_dataset = data.frame(cbind(unique(dataset[,1:5]), QRILC_df)) %>%
         pivot_longer(cols = 6:ncol(wider_dataset), names_to = "DTXSID", values_to = "Value")         

    return(imputed_dataset)

}

In [25]:
# will impute within each study that contains missing data
df_for_ids = mnar_df[is.na(mnar_df$Value),]
study_ids = unique(df_for_ids$HAWC_ID)

imputed_df = data.frame()
for (i in 1:length(study_ids)){

    filtered_df = preimputed_df %>%
        filter(HAWC_ID == study_ids[i])

    # calling fn
    imp_df = QRILC_imputation(filtered_df)
    imputed_df = rbind(imputed_df, imp_df)
    
}
# adding back in additional cols
joined_imputed_df = inner_join(unique(imputed_df), unique(weight_df[,1:8]))
joined_imputed_df = joined_imputed_df[,c(1:5,8,6,9,7)]

head(joined_imputed_df)

Joining with `by = join_by(HAWC_ID, Metric, Data, Study, Replicate, DTXSID)`


HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
914540,Weight,WS,Verma et al. 2021,1,PAH,DTXSID3024104,Fluoranthene,2.77
914540,Weight,WS,Verma et al. 2021,1,PAH,DTXSID6024254,Phenanthrene,3.72
914540,Weight,WS,Verma et al. 2021,2,PAH,DTXSID3024104,Fluoranthene,3.56
914540,Weight,WS,Verma et al. 2021,2,PAH,DTXSID6024254,Phenanthrene,2.14
914540,Weight,WS,Verma et al. 2021,3,PAH,DTXSID3024104,Fluoranthene,2.00
914540,Weight,WS,Verma et al. 2021,3,PAH,DTXSID6024254,Phenanthrene,0.91


In [26]:
imputed_weight_df = rbind(joined_imputed_df, 
                          # adding back in non-imputed data
                          preimputed_df %>%
                              filter(HAWC_ID %notin% study_ids),
                          # adding back in mar data
                          mar_weight_df)

head(imputed_weight_df)
dim(weight_df)
dim(imputed_weight_df)

HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
914540,Weight,WS,Verma et al. 2021,1,PAH,DTXSID3024104,Fluoranthene,2.77
914540,Weight,WS,Verma et al. 2021,1,PAH,DTXSID6024254,Phenanthrene,3.72
914540,Weight,WS,Verma et al. 2021,2,PAH,DTXSID3024104,Fluoranthene,3.56
914540,Weight,WS,Verma et al. 2021,2,PAH,DTXSID6024254,Phenanthrene,2.14
914540,Weight,WS,Verma et al. 2021,3,PAH,DTXSID3024104,Fluoranthene,2.00
914540,Weight,WS,Verma et al. 2021,3,PAH,DTXSID6024254,Phenanthrene,0.91


[1] 1053    9

[1] 1053    9

Double checking that when the data is split based on the `Class` and transformed into a wide format, we have at least 50% of the records with numeric values for each metal or chemical.

In [29]:
unique(imputed_weight_df$Chemical_Class)

[1] "PAH"   "Metal"

In [30]:
split_weight_df = imputed_weight_df %>%
    group_by(Chemical_Class) %>%
    group_split

metal_weight_df = split_weight_df[[1]]
pah_weight_df = split_weight_df[[2]]

head(metal_weight_df)

HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
1098462,Weight,WS,Niu et al. 2023,1,Metal,Al,Al,1.72
1098462,Weight,WS,Niu et al. 2023,1,Metal,Ca,Ca,1.65
1098462,Weight,WS,Niu et al. 2023,1,Metal,Cr,Cr,0.19
1098462,Weight,WS,Niu et al. 2023,1,Metal,Cu,Cu,0.43
1098462,Weight,WS,Niu et al. 2023,1,Metal,Fe,Fe,1.32
1098462,Weight,WS,Niu et al. 2023,1,Metal,K,K,174.21


In [35]:
wider_metal_weight_df = metal_weight_df %>%
    # creating sample ids
    unite("SampleID", c(colnames(metal_weight_df)[c(1,3,6)]), remove = FALSE) %>%
    # averaging to get rid of replicates
    group_by(HAWC_ID, Name) %>%
    mutate(Avg_Value = mean(Value)) %>%
    ungroup() %>%
    select(-c("Replicate", "Value", "Name")) %>%
    unique() %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value") 
wider_pah_weight_df = pah_weight_df %>%
    # creating sample ids
    unite("SampleID", c(colnames(pah_weight_df)[c(1,3,6)]), remove = FALSE) %>%
    # averaging to get rid of replicates
    group_by(HAWC_ID, Name) %>%
    mutate(Avg_Value = mean(Value)) %>%
    ungroup() %>%
    select(-c("Replicate", "Value", "Name")) %>%
    unique() %>%
    pivot_wider(names_from = "DTXSID", values_from = "Avg_Value") 

colSums(wider_metal_weight_df[,7:ncol(wider_metal_weight_df)])
colSums(wider_pah_weight_df[,7:ncol(wider_pah_weight_df)])

Al Ca Cr Cu Fe  K Mg Mn Ni Zn  B 
NA NA NA NA NA NA NA NA NA NA NA

DTXSID3024104 DTXSID6024254 DTXSID4020878 DTXSID0047741 DTXSID4075455 
           NA            NA            NA            NA            NA 
DTXSID4075459 DTXSID6062591 DTXSID8052691 DTXSID9059757 DTXSID2060383 
           NA            NA            NA            NA            NA 
DTXSID8074819 DTXSID4020161 DTXSID1025649 
           NA            NA            NA

# RF Imputation

In [53]:
random_forest_imputation = function(dataset){
    # ADD WORDS
    wider_dataset = dataset %>%
        # removing these cols temporarily 
        select(-c("Chemical_Class", "Name")) %>%
        pivot_wider(names_from = DTXSID, values_from = Value)
    
    preimputed_df = wider_dataset[,6:ncol(wider_dataset)]
    
    imputed_RF_object = missForest(as.matrix(preimputed_df))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(wider_dataset[,1:5], imputed_RF_df)) %>%
        pivot_longer(cols = 6:ncol(wider_dataset), names_to = "DTXSID", values_to = "Value")
    
    return(imputed_final_df)
}

# calling fn
rf_imputed_df = random_forest_imputation(imputed_weight_df %>% 
                             filter(HAWC_ID == unique(mar_weight_df$HAWC_ID)))
# adding back in additional cols
rf_imputed_df = inner_join(rf_imputed_df, unique(weight_df[,1:8]))
rf_imputed_df = rf_imputed_df[,c(1:5,9,6,8,7)]

head(rf_imputed_df)

Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response h

HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
279220,Weight,WS,Kasurinen et al. 2017,1,Metal,Ca,Ca,980.0
279220,Weight,WS,Kasurinen et al. 2017,1,Metal,Cr,Cr,74.0
279220,Weight,WS,Kasurinen et al. 2017,1,Metal,Cu,Cu,360.0
279220,Weight,WS,Kasurinen et al. 2017,1,Metal,Fe,Fe,340.0
279220,Weight,WS,Kasurinen et al. 2017,1,Metal,K,K,12000.0
279220,Weight,WS,Kasurinen et al. 2017,1,Metal,Mn,Mn,275.9


In [54]:
# combining data
final_df = full_join(imputed_weight_df %>% filter(HAWC_ID != unique(mar_weight_df$HAWC_ID)),
                            rf_imputed_df)

head(final_df)
dim(final_df)

Joining with `by = join_by(HAWC_ID, Metric, Data, Study, Replicate,
Chemical_Class, DTXSID, Name, Value)`


HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID3021774,Acenaphthene,132.7
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID4075459,Benzo(c)phenanthrene,493.9
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID8052691,Benzo(j)fluoranthene,617.3
271638,Weight,WS,Jalava et al. 2012,1,Metal,Ca,Ca,1140.0
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID5047740,Coronene,75.5
271638,Weight,WS,Jalava et al. 2012,1,PAH,DTXSID6062591,Cyclopenta(cd)pyrene,1067.0


[1] 1248    9

In [55]:
head(imputed_weight_df)

HAWC_ID,Metric,Data,Study,Replicate,Chemical_Class,DTXSID,Name,Value
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
914540,Weight,WS,Verma et al. 2021,1,PAH,DTXSID3024104,Fluoranthene,2.77
914540,Weight,WS,Verma et al. 2021,1,PAH,DTXSID6024254,Phenanthrene,3.72
914540,Weight,WS,Verma et al. 2021,2,PAH,DTXSID3024104,Fluoranthene,3.56
914540,Weight,WS,Verma et al. 2021,2,PAH,DTXSID6024254,Phenanthrene,2.14
914540,Weight,WS,Verma et al. 2021,3,PAH,DTXSID3024104,Fluoranthene,2.00
914540,Weight,WS,Verma et al. 2021,3,PAH,DTXSID6024254,Phenanthrene,0.91


In [ ]:
# # exporting
# write.xlsx(final_df, paste0(Output,"/", "Processed_Weight_Data_", cur_date, ".xlsx"), 
#            rowNames = FALSE)
# write.xlsx(vol_df, paste0(Output,"/", "Processed_Volume_Data_", cur_date, ".xlsx"), 
#            rowNames = FALSE)